## exact same with Pytorch

#### Happenings below:

- We are defining the variables as `requires_grad=True` so that PyTorch knows that it needs to calculate the gradients for these variables. It's set to `False` by default.
- We are defining the operations as `torch` operations.
- The `.double()` method is used to convert the variables to `float64`. This is because Python uses `float64` by default, but the default element datatype for `torch.Tensor()` is `float32`. We want to make sure that we are comparing apples to apples.
- The `.item()` method is used to get the actual value of the tensor. This is because the tensor itself is a wrapper around the actual value, and we need to get the actual value to compare it with our implementation.
- Torch already has its own `backward()` function, so we don't need to implement it ourselves. We can just call `loss.backward()` and it will calculate the gradients for us.


In [1]:
import torch

x1 = torch.Tensor([2.0]).double();x1.requires_grad=True
x2 = torch.Tensor([0.0]).double();x2.requires_grad=True
w1 = torch.Tensor([-3.0]).double();w1.requires_grad=True
w2 = torch.Tensor([1.0]).double();w2.requires_grad=True
b=torch.Tensor([6.8813]).double();b.requires_grad=True

n=x1*w1 + x2*w2 +b
o=torch.tanh(n)

print('o:', o.item())
o.backward()

print('x2:', x2.grad.item())
print('w2:', w2.grad.item())
print('x1:', x1.grad.item())
print('w1:', w1.grad.item())

o: 0.7070699720278941
x2: 0.5000520546564731
w2: 0.0
x1: -1.5001561639694192
w1: 1.0001041093129461


In [6]:
import random

class Value:
    def __init__(self, data):
        self.data = data
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data * other.data)
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data + other.data)
    def tanh(self):
        import math
        return Value((math.exp(2*self.data)-1)/(math.exp(2*self.data)+1))
    def __repr__(self):
        return f"Value(data={self.data})"

## Building a MLP in Micrograd

### satrting with a single individual neuron

#### index
- nin =number of inputs
- nout =number of outputs
- w = weights
- b = bias

In [11]:
class Neuron:
    def __init__(self, nin):
        self.w= [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b= Value(random.uniform(-1,1))

    def __call__(self,x):
        #w*x+b
        act=sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out= act.tanh()
        return out
    
class Layer:
    def __init__(self, nin,nout):
        self.neurons=[Neuron(nin) for _ in range(nout)]

    def __call__(self,x):
        outs=[n(x) for n in self.neurons]
        return outs

class MLP:  # instead of single input taking list of inputs, the list defines the size of all layer in our mlp
    def __init__(self, nin , nouts):
        sz=[nin]+nouts
        self.layers=[Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x=layer(x)
        return x

x=[2.0,3.0, -1.0]
n=MLP(3,[4,4,1])
n(x)

[Value(data=-0.29220264597018003)]